# Notes
The data generated by this notebook can be used to make the maps in figure 1 and is saved in two locations:
* In the `main` folder under the path pointed to by `FIGURE_PATH`
* In the `d3map` folder in the folder this notebook is in - this makes generating the map from that folder easier.

In [1]:
import sys
sys.executable

'/opt/anaconda3/bin/python'

In [2]:
%matplotlib inline
import sys
import os
DATA_PATH = os.getenv('DATA_PATH')
CODE_PATH = os.getenv('CODE_PATH')
FIGURE_PATH = os.getenv('FIGURE_PATH')

sys.path.insert(0, os.path.join(CODE_PATH))

import pandas as pd
import numpy as np
import json
import re

import time

from src.load import EGRID, BA_DATA

from d3map_utils import resetCoords, addDataNodes, data_path, data_path2 


import matplotlib.pyplot as plt

import logging.config
logging.config.fileConfig(os.path.join(CODE_PATH, "src/logging.conf"))
logger = logging.getLogger(__name__)

In [3]:
data_path

'd3map/data'

In [4]:
# Make sure directory exists
os.makedirs(data_path2, exist_ok=True)

In [5]:
co2 = BA_DATA(fileNm=os.path.join(DATA_PATH, "analysis/SEED_CO2_Y.csv"), variable="CO2")
so2 = BA_DATA(fileNm=os.path.join(DATA_PATH, "analysis/SEED_SO2_Y.csv"), variable="SO2")
nox = BA_DATA(fileNm=os.path.join(DATA_PATH, "analysis/SEED_NOX_Y.csv"), variable="NOX")


In [6]:
WECC_BAs = [
    "AVA", "AZPS","BANC", "BPAT","CHPD",
    "CISO", "DEAA", "DOPD", "EPE", "GCPD", "GRMA", "GWA",
    "HGMA", "IID", "IPCO", "LDWP", "NEVP", "NWMT",
    "PACE", "PACW", "PGE", "PNM", "PSCO",
    "PSEI", "SCL", "SRP", "TEPC","TIDC",
    "TPWR", "WACM", "WALC", "WAUW", "WWA"]

In [7]:
# CO2 map
resetCoords()

for poll in ["CO2", "SO2", "NOX"]:
    for variable in ["E", poll]:
        
        variable_scalings = {
            "E": 1e-6,
            "CO2": 1e-6,
            "SO2": 1e-3,
            "NOX": 1e-3,
        }
        polli_scalings = {
            "CO2": 1000,
            "SO2": 1e6,
            "NOX": 1e6,
        }
        legCircleTitles = {
            "E": "Electricity consumption (TWh)",
            "CO2": "Carbon consumption (Mtons)",
            "SO2": "SO2 consumption (ktons)",
            "NOX": "NOX consumption (ktons)"
        }
        legLineTitles = {
            "E": "Electricity trade (TWh)",
            "CO2": "Carbon trade (Mtons)",
            "SO2": "SO2 trade (ktons)",
            "NOX": "NOx trade (ktons)",
        }
        legColorTitles = {
            "CO2": "Consumption-based carbon intensity (kg/MWh)",
            "SO2": "Consumption-based SO2 intensity (ton/MWh)",
            "NOX": "Consumption-based NOX intensity (ton/MWh)",
        }
        titles = {
            "E": "ELECTRICITY",
            "CO2": "CARBON",
            "SO2": "SULFUR DIOXIDE",
            "NOX": "NITROGEN OXIDES"
        }
        units =  {
            "E": "TWh",
            "CO2": "Mtons",
            "SO2": "ktons",
            "NOX": "ktons"
        }

        # Unpack options
        legCircleTitle = legCircleTitles[variable]
        legLineTitle = legLineTitles[variable]
        variable_scaling = variable_scalings[variable]
        unit = units[variable]
        title = titles[variable]
        legColorTitle = legColorTitles[poll]
        polli_scaling = polli_scalings[poll]
        fileNm_out = "graph_%s_%si.json" % (variable, poll)

        poll_data = BA_DATA(fileNm=os.path.join(
                DATA_PATH, "analysis/SEED_%s_Y.csv" % poll),
                            variable="%s" % poll)
        elec = BA_DATA(fileNm=os.path.join(
                DATA_PATH, "analysis/SEED_E_Y.csv"),
                       variable="E")

        if variable == "E":
            variable_data = elec
        else:
            variable_data = poll_data

        # Prepare data
        D = (variable_data.df[variable_data.get_cols(field="D")].values[0]
             * variable_scaling)
        X = (poll_data.df[poll_data.get_cols(field="D")].values[0]
             / elec.df[elec.get_cols(field="D")].values[0])
        

        polli_data = dict(zip(elec.regions, polli_scaling * X))
        elec_data = dict(zip(elec.regions, D))

        baseGraphPath = os.path.join(data_path, "graph2.json")
        with open(baseGraphPath, 'r') as fr:
            graph = json.load(fr)

        data = elec_data
        for ba in elec.regions:
            el = data.pop(ba)
            if el == 0.:
                el = None
            data[ba] = el
        graph = addDataNodes(graph, "E_D", data)
        graph = addDataNodes(graph, "E_D", data, "labels")

        data = polli_data
        for ba in elec.regions:
            el = data.pop(ba)
            if np.isnan(el):
                el = None
            data[ba] = el
        graph = addDataNodes(graph, "CO2_Di", data)
        
        data = {}
        for ba in elec.regions:
            if ba in WECC_BAs:
                data[ba] = "wecc"
            elif ba == "ERCO":
                data[ba] = "erco"
            else:
                data[ba] = "eic"
        graph = addDataNodes(graph, "interconnect", data)

        node_list = [el["shortNm"] for el in graph['nodes']]
        shortNm2ind = {node["shortNm"]:i for i, node in enumerate(graph['nodes'])}


        # Add data for the links
        links = []

        regions = variable_data.regions

        for i in range(len(regions)):
            for j in range(i, len(regions)):
                from_ba = regions[i]
                to_ba = regions[j]
                if ((elec.KEY["ID"] % (from_ba, to_ba) in elec.df.columns)
                        & (elec.KEY["ID"] % (to_ba, from_ba) in elec.df.columns)):
                    elec_transfer = elec.df.loc[
                        :, elec.KEY["ID"] % (from_ba, to_ba)].values[0]
                    
                    if elec_transfer < 0: # Have this be positive
                        from_ba = regions[j]
                        to_ba = regions[i]
                        elec_transfer = elec.df.loc[
                            :, elec.KEY["ID"] % (from_ba, to_ba)].values[0]
                    
                    if variable != "E":
                        elec_transfer *= (polli_data[from_ba] / polli_scaling)
                    links += [{
                            "source": shortNm2ind[from_ba],
                            "target": shortNm2ind[to_ba],
                            "TI": elec_transfer*variable_scaling,
                            "TI_i": polli_data[from_ba]}]

        graph["links"] = links


        graph["meta"] = {
            "colorModeAuto": False,
            "fieldRadius": 'E_D',
            "fieldLineWidth": 'TI',
            "fieldCircle": 'CO2_Di',
            "fieldLineColor": 'TI_i',
            "legColorTitle": legColorTitle,
            "legCircleTitle": legCircleTitle,
            "legLineTitle": legLineTitle,
            "unit": unit,
            "title": title
        }

        graphPath_out = os.path.join(data_path, fileNm_out)
        with open(graphPath_out, 'w') as fw:
            json.dump(graph, fw)
        # Save a local copy
        graphPath_out = os.path.join(data_path2, fileNm_out)
        with open(graphPath_out, 'w') as fw:
            json.dump(graph, fw)

In [8]:
# Aside: Understanding CA imports of SOx and NOx
so2.df[so2.get_cols(r="CISO", field="NG")+ so2.get_cols(r="CISO", field="D")]

,SO2_CISO_NG,SO2_CISO_D
2016-01-01,2118.848568,8735.191021


In [9]:
so2.df.loc[:, [so2.KEY["ID"] % (from_ba, "CISO") for from_ba in so2.get_trade_partners("CISO")]]

,SO2_AZPS-CISO_ID,SO2_BANC-CISO_ID,SO2_BPAT-CISO_ID,SO2_IID-CISO_ID,SO2_LDWP-CISO_ID,SO2_NEVP-CISO_ID,SO2_PACW-CISO_ID,SO2_SRP-CISO_ID,SO2_TIDC-CISO_ID,SO2_WALC-CISO_ID
2016-01-01,1035.953807,151.857912,1162.680019,373.917347,2518.789281,-27.437312,19.022265,1218.767239,-33.66503,196.456926


In [10]:
so2.df[so2.get_cols(field="D")].transpose().sort_values(by="2016-01-01 00:00:00", ascending=False)

,2016-01-01 00:00:00
SO2_MISO_D,420964.367648
SO2_PJM_D,315835.025727
SO2_ERCO_D,181045.348412
SO2_SWPP_D,165738.256220
SO2_TVA_D,68401.719363
SO2_SOCO_D,42015.586451
SO2_PACE_D,30623.108411
SO2_AECI_D,21440.502109
SO2_LGEE_D,20211.165984
SO2_FPC_D,14792.060227


In [11]:
nox.df.loc[:, [nox.KEY["ID"] % (from_ba, "CISO") for from_ba in so2.get_trade_partners("CISO")]]

,NOX_AZPS-CISO_ID,NOX_BANC-CISO_ID,NOX_BPAT-CISO_ID,NOX_IID-CISO_ID,NOX_LDWP-CISO_ID,NOX_NEVP-CISO_ID,NOX_PACW-CISO_ID,NOX_SRP-CISO_ID,NOX_TIDC-CISO_ID,NOX_WALC-CISO_ID
2016-01-01,3112.918929,366.353885,1755.627849,100.829905,7356.817784,-184.249044,52.580377,4889.135917,-226.069875,588.856287


In [12]:
nox.df[nox.get_cols(r="CISO", field="NG")+ nox.get_cols(r="CISO", field="D")]

,NOX_CISO_NG,NOX_CISO_D
2016-01-01,40846.392096,58659.19411
